In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
%matplotlib inline
#%matplotlib notebook
sns.set(rc={'figure.figsize':(11.7,8.27)})
from scipy import stats
import statsmodels.api as sm
from scipy.stats import ks_2samp
import scipy.stats as stats
from scipy.stats import ttest_rel

In [3]:
df_tesco = pd.read_pickle(r'df_tesco_marker_V3.pkl')

cols_sel = pd.read_excel(r'cols_select.xlsx')

df = df_tesco.loc[:,cols_sel.query('is_sel ==1')['colName'].tolist()]

# Rank the median sales for exclusive shopsaleas according to the day of the week # 
df['weekday'] = df['Transaction Date'].dt.strftime('%A')

# weekday_sales = df.groupby(['Site Name','weekday'])\
#                     ['exclShopSales'].agg({'median':'median'}).unstack().reset_index()

# weekday_sales.columns = [' '.join(col).strip() for col in weekday_sales.columns.values]

In [4]:
df.columns

Index(['Site Number', 'Site Name', 'Transaction Date',
       'Sales Baskets With Fuel', 'PBL', 'volumeADO', 'volumeUL95',
       'sales_valUL95', 'totalSalesEsso', 'UL 95_ Approved Price',
       'UL 95_ Marker Price', 'UL 95_ Approved Price Vs Marker Price',
       'ADO_ Approved Price', 'ADO_ Marker Price',
       'ADO_ Approved Price Vs Marker Price', 'exclShopSales',
       'pctChgExclShopSalesFromMedian', 'shopSalesFromFuel',
       'pctChgShopSalesFromFuelFromMedian', 'pctChgFuelSalesFromMedian',
       'log_exclShopSales', 'log_shopSalesFromFuel', 'logP_by_P0_ADO',
       'logP_by_P0_UL95', 'weekday'],
      dtype='object')

In [5]:
df['shopSalesRatio'] = round((df['Sales Baskets With Fuel']-df['totalSalesEsso'])/df['totalSalesEsso'],2)

In [7]:
df['shopSalesRatio'].describe()

count   61,990.00
mean         0.13
std          0.04
min         -0.10
25%          0.10
50%          0.13
75%          0.15
max          0.50
Name: shopSalesRatio, dtype: float64

In [5]:
df.to_excel(r'tab_viz_elasticty_2.xlsx')

In [8]:
df_med_shopSalesR = df.groupby(['Site Name','PBL'])['shopSalesRatio'].agg({'median_shopSalesR':'median'}).reset_index()
q_labels = ['Low','Medium','High','Very High']
df_med_shopSalesR['shopSalesR_q'] = pd.qcut(df_med_shopSalesR["median_shopSalesR"],q=4,labels=q_labels)

In [9]:
df_med_shopSalesR.to_excel('shopSalesToFuelSalesRationQ.xlsx')

In [12]:
df_med_shopSalesR.head()

Site Name        PBL  median_shopSalesR shopSalesR_q
0  ABBEY ST ACCRNGTN EXP 200,911.00               0.12       Medium
1       ABINGTON EXPRESS 200,740.00               0.14         High
2  ACKLAM RD MIDSBRO EXP 328,841.00               0.08          Low
3          ACTON EXPRESS 300,931.00               0.18    Very High
4     ADDISCOMBE EXPRESS 201,307.00               0.14         High

In [109]:
df['weekDayMedian'] = df.groupby(['Site Name','weekday'])['exclShopSales'].transform('median')
df['weekDayMedianRank'] = df.groupby(['Site Name'])['weekDayMedian'].rank(ascending = 0, method = 'dense')

weekday_sales_2 = df.groupby(['Site Name','PBL','weekday'])\
                    ['weekDayMedianRank'].agg({'medianSales':'median'}).unstack().reset_index()
weekday_sales_2.columns = [''.join(col).strip() for col in weekday_sales_2.columns.values]

df['weekDayMedian'] = df.groupby(['Site Name','weekday'])['shopSalesFromFuel'].transform('median')
df['weekDayMedianRank'] = df.groupby(['Site Name'])['weekDayMedian'].rank(ascending = 0, method = 'dense')

weekday_sales_3 = df.groupby(['Site Name','PBL','weekday'])\
                    ['weekDayMedianRank'].agg({'medianSales':'median'}).unstack().reset_index()
weekday_sales_3.columns = [''.join(col).strip() for col in weekday_sales_3.columns.values]

df['weekDayMedian'] = df.groupby(['Site Name','weekday'])['volumeADO'].transform('median')
df['weekDayMedianRank'] = df.groupby(['Site Name'])['weekDayMedian'].rank(ascending = 0, method = 'dense')

weekday_sales_4 = df.groupby(['Site Name','PBL','weekday'])\
                    ['weekDayMedianRank'].agg({'medianSales':'median'}).unstack().reset_index()
weekday_sales_4.columns = [''.join(col).strip() for col in weekday_sales_4.columns.values]

df['weekDayMedian'] = df.groupby(['Site Name','weekday'])['volumeUL95'].transform('median')
df['weekDayMedianRank'] = df.groupby(['Site Name'])['weekDayMedian'].rank(ascending = 0, method = 'dense')

weekday_sales_5 = df.groupby(['Site Name','PBL','weekday'])\
                    ['weekDayMedianRank'].agg({'medianSales':'median'}).unstack().reset_index()
weekday_sales_5.columns = [''.join(col).strip() for col in weekday_sales_5.columns.values]

weekday_sales_2['label']= 'exclShopSales'
weekday_sales_3['label']= 'shopSalesFromFuel'
weekday_sales_4['label'] = 'salesVolADO'
weekday_sales_5['label'] = 'salesVolUL95'

wk_sales = pd.concat([weekday_sales_2,weekday_sales_3,weekday_sales_4,weekday_sales_5],axis=0,ignore_index=True)

In [117]:
PDI_elasticity_results = pd.read_excel(r'pdi_elaticity_values.xlsx')
pdi = PDI_elasticity_results.pivot_table(index=['DisplayId'],columns='Grade',values='Elasticity').reset_index()
q_labels = ['Very High','High','Medium','Low','Very Low']
pdi['El_ADO_quantile'] = pd.qcut(pdi["ADO"],q=5,labels=q_labels)
pdi['El_UL_quantile'] = pd.qcut(pdi["UL 95"],q=5,labels=q_labels)
weekday_sales = wk_sales.merge(pdi,how = 'left',left_on='PBL',right_on='DisplayId')

In [118]:
weekday_sales.to_excel('sales_weekday_ranking.xlsx',index=False)